# Aritifical Neural Network for PBL

Prediction of the secondary structure of a protein using an ANN in order to predict the states of every amino acid in the sequence. This then gives an overview on how a protein could look like in reality only given the sequence of amino acids.

In [1]:
import numpy as num
import pandas as pnds
import math
import sys
import datetime
import sklearn.metrics as metrics

from joblib import dump, load

from sklearn import datasets
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, GridSearchCV

# Import of data

Import data from text files.

In [2]:
# open files
file_test_sequence = open("/home/annette/Data/Studium/Module/PBL/Dataset/test_sequences.fasta")
file_training_sequence = open("/home/annette/Data/Studium/Module/PBL/Dataset/training_sequences.fasta")
file_test_structure = open("/home/annette/Data/Studium/Module/PBL/Dataset/test_structure.fasta")
file_training_structure = open("/home/annette/Data/Studium/Module/PBL/Dataset/training_structure.fasta")
file_test_disorder = open("/home/annette/Data/Studium/Module/PBL/Dataset/test_pdb_disorder.fasta")
file_training_disorder = open("/home/annette/Data/Studium/Module/PBL/Dataset/training_disorder.fasta")

# delete \n from file
list_test_sequence = file_test_sequence.readlines()
for number in range(len(list_test_sequence)):
    element = list_test_sequence[number]
    list_test_sequence[number] = element[0:len(element) - 1]
    
list_training_sequence = file_training_sequence.readlines()
for number in range(len(list_training_sequence)):
    element = list_training_sequence[number]
    list_training_sequence[number] = element[0:len(element) - 1]

list_test_structure = file_test_structure.readlines()
for number in range(len(list_test_structure)):
    element = list_test_structure[number]
    list_test_structure[number] = element[0:len(element) - 1]
    
list_training_structure = file_training_structure.readlines()
for number in range(len(list_training_structure)):
    element = list_training_structure[number]
    list_training_structure[number] = element[0:len(element) - 1]
    
list_test_disorder = file_test_disorder.readlines()
for number in range(len(list_test_disorder)):
    element = list_test_disorder[number]
    list_test_disorder[number] = element[0:len(element) - 1]
    
list_training_disorder = file_training_disorder.readlines()
for number in range(len(list_training_disorder)):
    element = list_training_disorder[number]
    list_training_disorder[number] = element[0:len(element) - 1]

# close files
file_test_sequence.close()
file_training_sequence.close()
file_test_structure.close()
file_training_structure.close()
file_test_disorder.close()
file_training_disorder.close()

# Preparation of data

Change from given 8 states to needed 3 states

labels:
* sheet: 0
* coil: 1
* helix: 2

Test set

In [3]:
# Change to 3 states
for index_list, prot_sequ in enumerate(list_test_structure):
    
    if prot_sequ[0:1] == ">" or len(prot_sequ) == 0:
        continue
        
    test_structure = [] 
    for index in range(len(prot_sequ)):
        if prot_sequ[index] == ' ':
            if list_test_disorder[index_list][index] == '-':
                test_structure.append("C")
            else:
                test_structure.append(" ")
        elif prot_sequ[index] == 'H':
            test_structure.append("H")
        elif prot_sequ[index] == 'E':
            test_structure.append("E")
        elif prot_sequ[index] == 'T':
            test_structure.append("C")
        elif prot_sequ[index] == 'S':
            test_structure.append("C")
        elif prot_sequ[index] == 'G':
            test_structure.append("H")
        elif prot_sequ[index] == 'B':
            test_structure.append("E")
        elif prot_sequ[index] == 'I':
            test_structure.append("H")
        elif prot_sequ[index] == 'C':
            test_structure.append("C")
            
    structure3_test = ''.join(test_structure)
    list_test_structure[index_list] = structure3_test

print("Ready.")

list_test_labels = []

# Change to numbers = labels
for index_list, prot_sequ in enumerate(list_test_structure):

    if prot_sequ[0:1] == ">" or len(prot_sequ) == 0:
        list_test_labels.append(prot_sequ)
        continue

    test_structure = []
    for index in range(len(prot_sequ)):
        if prot_sequ[index] == 'H':
            test_structure.append("2")
        elif prot_sequ[index] == 'E':
            test_structure.append("0")
        elif prot_sequ[index] == ' ':
            test_structure.append(" ")
        elif prot_sequ[index] == 'C':
            test_structure.append("1")
    
    labels_test = ''.join(test_structure)
    list_test_labels.append(labels_test)

print("Ready.")

Ready.
Ready.


Training set

In [4]:
# Change to 3 states
for index_list, prot_sequ in enumerate(list_training_structure):
     
    if prot_sequ[0:1] == ">" or len(prot_sequ) == 0:
        continue
     
    training_structure = []
    for index in range(len(prot_sequ)):
        if prot_sequ[index] == ' ':
            if list_training_disorder[index_list][index] == '-':
                training_structure.append("C")
            else:
                training_structure.append(" ")
        elif prot_sequ[index] == 'H':
            training_structure.append("H")
        elif prot_sequ[index] == 'E':
            training_structure.append("E")
        elif prot_sequ[index] == 'T':
            training_structure.append("C")
        elif prot_sequ[index] == 'S':
            training_structure.append("C")
        elif prot_sequ[index] == 'G':
            training_structure.append("H")
        elif prot_sequ[index] == 'B':
            training_structure.append("E")
        elif prot_sequ[index] == 'I':
            training_structure.append("H")
        elif prot_sequ[index] == 'C':
            training_structure.append("C")

    
    structure3_training = ''.join(training_structure)
    list_training_structure[index_list] = structure3_training

print("Ready.")
# print(list_training_structure)


list_training_labels = []

# Change to numbers = labels
for index_list, prot_sequ in enumerate(list_training_structure):

    if prot_sequ[0:1] == ">" or len(prot_sequ) == 0:
        list_training_labels.append(prot_sequ)
        continue

    training_structure = []
    for index in range(len(prot_sequ)):
        if prot_sequ[index] == 'H':
            training_structure.append("2")
        elif prot_sequ[index] == 'E':
            training_structure.append("0")
        elif prot_sequ[index] == ' ':
            training_structure.append(" ")
        elif prot_sequ[index] == 'C':
            training_structure.append("1")
    
    labels_training = ''.join(training_structure)
    list_training_labels.append(labels_training)

print("Ready.")

Ready.
Ready.


Connection of name, sequence and labels of protein into dictionary

In [5]:
# list_test_sequence. list_training_sequence, list_test_labels, list_training_labels
whole_data_test = {}
whole_data_training = {}

# test data
for index in range(0, len(list_test_sequence), 3):
    key = list_test_sequence[index]
    key = key[1:len(key)]
    value1 = list_test_sequence[index + 1]
    value2 = list_test_labels[index + 1]
    value = [value1, value2]
    whole_data_test[key] = value


# training data
for index in range(0, len(list_training_sequence), 3):
    key = list_training_sequence[index]
    key = key[1:len(key)]
    value1 = list_training_sequence[index + 1]
    value2 = list_training_labels[index + 1]
    value = [value1, value2]
    whole_data_training[key] = value

print("Ready.")

Ready.


# Input features

In [6]:
class_names = ['coil', 'sheet', 'helix']
helix_proba = {'A': 1.32, 'B': 0.86, 'C': 1.22, 'D': 0.86, 'E': 1.2, 'F': 1.02, 'G': 0.5, 'H': 0.9, 'I': 1.08, 'J': 1.2, 'K': 1.06, 'L': 1.32, 'M': 1.34, 'N': 0.86, 'O': 1., 'P': 0.3, 'Q': 1.24, 'R': 1.16, 'S': 0.84, 'T': 0.84, 'U': 1., 'V': 0.9, 'W': 1.06, 'X': 1., 'Y': 0.98, 'Z': 1.22}
sheet_proba = {'A': 0.78, 'B': 0.43, 'C': 1.36, 'D': 0.42, 'E': 0.64, 'F': 1.52, 'G': 0.64, 'H': 0.98, 'I': 1.78, 'J': 1.48, 'K': 0.7, 'L': 1.18, 'M': 1.06, 'N': 0.44, 'O': 0.96, 'P': 0.02, 'Q': 0.74, 'R': 0.86, 'S': 0.76, 'T': 1.04, 'U': 0.96, 'V': 1.92, 'W': 1.22, 'X': 0.96, 'Y': 1.1, 'Z': 0.69}
hydrophob = {'A': 1.8, 'B': -3.5, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4, 'H': -3.2, 'I': 4.5, 'J': 4.15, 'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5, 'O': -3., 'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8, 'T': -0.7, 'U': 1., 'V': 4.2, 'W': -0.9, 'X': -0.49, 'Y': -1.3, 'Z': -3.5}
isoelec_point = {'A': 6.1, 'B': 4.13, 'C': 5.05, 'D': 2.85, 'E': 3.22, 'F': 5.84, 'G': 5.97, 'H': 7.47, 'I': 5.94, 'J': 5.96, 'K': 9.59, 'L': 5.98, 'M': 5.74, 'N': 5.41, 'O': 7., 'P': 6.3, 'Q': 5.65, 'R': 11.76, 'S': 5.68, 'T': 5.6, 'U': 6., 'V': 5.96, 'W': 5.64, 'X': 6.07, 'Y': 5.66, 'Z': 4.44}
amino_acids = {'A': 0.1, 'C': 0.1, 'D': 0.1, 'E': 0.1, 'F': 0.1, 'G': 0.1, 'H': 0.1, 'I': 0.1, 'K': 0.1, 'L': 0.1, 'M': 0.1, 'N': 0.1, 'P': 0.1, 'Q': 0.1, 'R': 0.1, 'S': 0.1, 'T': 0.1, 'V': 0.1, 'W': 0.1, 'Y': 0.1, 'B': 0.2, 'J': 0.2, 'Z': 0.2, 'O': 0.4, 'U': 0.4, 'X': 0.6}


Normalization of input features

In [7]:
array = num.empty((5,26))

# helix
for index, aa in enumerate(helix_proba):
    array[0][index] = helix_proba.get(aa)
    
# sheet
for index, aa in enumerate(sheet_proba):
    array[1][index] = sheet_proba.get(aa)

# hydrophob
for index, aa in enumerate(hydrophob):
    array[2][index] = hydrophob.get(aa)

# isoelec
for index, aa in enumerate(isoelec_point):
    array[3][index] = isoelec_point.get(aa)

# amino acids
for index, aa in enumerate(amino_acids):
    array[4][index] = amino_acids.get(aa)

# Normalizer, StandardScaler
array_transformed = Normalizer().fit_transform(array)

# back to dictionary
# helix
for index, aa in enumerate(helix_proba):
    helix_proba[aa] = array_transformed[0][index]
print(helix_proba)

# sheet
for index, aa in enumerate(sheet_proba):
    sheet_proba[aa] = array_transformed[1][index]
print(sheet_proba)

# hydrophob
for index, aa in enumerate(hydrophob):
    hydrophob[aa] = array_transformed[2][index]
print(hydrophob)

# isoelec
for index, aa in enumerate(isoelec_point):
    isoelec_point[aa] = array_transformed[3][index]
print(isoelec_point)

# amino acids
for index, aa in enumerate(amino_acids):
    amino_acids[aa] = array_transformed[4][index]
print(amino_acids)

{'A': 0.24934615190204282, 'B': 0.1624527959361794, 'C': 0.23045629190946382, 'D': 0.1624527959361794, 'E': 0.226678319910948, 'F': 0.19267657192430582, 'G': 0.094449299962895, 'H': 0.170008739933211, 'I': 0.2040104879198532, 'J': 0.226678319910948, 'K': 0.20023251592133742, 'L': 0.24934615190204282, 'M': 0.2531241239005586, 'N': 0.1624527959361794, 'O': 0.18889859992579, 'P': 0.056669579977737, 'Q': 0.2342342639079796, 'R': 0.2191223759139164, 'S': 0.1586748239376636, 'T': 0.1586748239376636, 'U': 0.18889859992579, 'V': 0.170008739933211, 'W': 0.20023251592133742, 'X': 0.18889859992579, 'Y': 0.1851206279272742, 'Z': 0.23045629190946382}
{'A': 0.14730044172548845, 'B': 0.08120408966917952, 'C': 0.2568315394188004, 'D': 0.07931562246757069, 'E': 0.12086190090296488, 'F': 0.2870470146445416, 'G': 0.12086190090296488, 'H': 0.18506978575766497, 'I': 0.33614716188637106, 'J': 0.27949314583810625, 'K': 0.13219270411261783, 'L': 0.22283912978984147, 'M': 0.2001775233705356, 'N': 0.08309255687

# Generating the input vectors and the target array

Windows of sequences: not if space for middle AA or not one from 20 letters, else: 7 on each side

structure: for every AA: value of
* amino_acids
* hydrophob
* isoelec_point
* helix_proba
* sheet_proba

Test set

In [8]:
vectors_test = []
target_test_list = []
counter_h = 0
counter_e = 0
counter_c = 0

for name in whole_data_test:
    data = whole_data_test.get(name)
    sequence = data[0]
    structure = data[1]
    counter_amino_acids = 0
    target_test = []
    
    for aa in range(len(sequence)):
        if sequence[aa] == ('B' or 'J' or 'O' or 'U' or 'X' or 'Z'):
            continue
            
        if structure[aa] == ' ':
            continue
                                
        if structure[aa] =='2':
            counter_h = counter_h + 1
        elif structure[aa] == '0':
            counter_e = counter_e + 1
        elif structure[aa] == '1':
            counter_c = counter_c + 1

        counter_amino_acids = counter_amino_acids + 1
        vector = []
        window = []
        if aa < 7:
            for index in range(0, 7 - aa):
                window.append(sequence[0])
            for index in range(0, aa + 8):
                window.append(sequence[index])
        elif aa > len(sequence) - 8:
            for index in range(aa - 7, len(sequence)):
                window.append(sequence[index])
            for index in range(len(window), 15):
                window.append(sequence[len(sequence) - 1])
        else:
            for index in range(aa - 7, aa + 8):
                window.append(sequence[index])
        
        # Generation the target array
        target_test_list.append(structure[aa])
        target_test.append(structure[aa])
        
        for acid in window:
            vector.append(amino_acids.get(acid))
            vector.append(hydrophob.get(acid))
            vector.append(isoelec_point.get(acid))
            vector.append(helix_proba.get(acid))
            vector.append(sheet_proba.get(acid))
            
        vectors_test.append(vector)
        
    whole_data_test[name].append(str(counter_amino_acids))
    whole_data_test[name].append(target_test)
        
input_vectors_test = num.array(vectors_test, dtype=object)
for index in range(len(input_vectors_test)):
        vector = input_vectors_test[index]
        input_vectors_test[index] = num.array(vector, dtype=object)
target_test = num.array(target_test_list)
print(counter_h)
print(counter_e)
print(counter_c)
print(86440)
print("Ready.")

27195
19601
33762
86440
Ready.


Training set

In [9]:
vectors_training = []
target_training_list = []
counter_h = 0
counter_e = 0
counter_c = 0

counter = 0

for name in whole_data_training:
    data = whole_data_training.get(name)
    sequence = data[0]
    structure = data[1]
    counter_amino_acids = 0
    target_training = []
    
    for aa in range(len(sequence)):
        if sequence[aa] == 'B' or sequence[aa] == 'J' or sequence[aa] == 'O' or sequence[aa] == 'U' or sequence[aa] == 'X' or sequence[aa] == 'Z':
            continue

        if structure[aa] == ' ':
            continue
                
        if structure[aa] =='2':
            counter_h = counter_h + 1
        elif structure[aa] == '0':
            counter_e = counter_e + 1
        elif structure[aa] == '1':
            counter_c = counter_c + 1

        counter_amino_acids = counter_amino_acids + 1
        vector = []
        window = []
        if aa < 7:
            for index in range(0, 7 - aa):
                window.append(sequence[0])
            for index in range(0, aa + 8):
                window.append(sequence[index])
        elif aa > len(sequence) - 8:
            for index in range(aa - 7, len(sequence)):
                window.append(sequence[index])
            for index in range(len(window), 15):
                window.append(sequence[len(sequence) - 1])
        else:
            for index in range(aa - 7, aa + 8):
                window.append(sequence[index])
        
        # Generation the target array
        target_training_list.append(structure[aa])
        target_training.append(structure[aa])
        
        for acid in window:
            vector.append(amino_acids.get(acid))
            vector.append(hydrophob.get(acid))
            vector.append(isoelec_point.get(acid))
            vector.append(helix_proba.get(acid))
            vector.append(sheet_proba.get(acid))

        if pnds.isnull(num.array(vector).all()):
            print(name)
            print(aa)
            print(sequence[aa])
            counter = counter + 1
            
        vectors_training.append(vector)
        
    whole_data_training[name].append(str(counter_amino_acids))
    whole_data_training[name].append(target_training)
        
input_vectors_training = num.array(vectors_training, dtype=object)
for index in range(len(input_vectors_training)):
        vector = input_vectors_training[index]
        input_vectors_training[index] = num.array(vector, dtype=object)
target_training = num.array(target_training_list)
print(counter_h)
print(counter_e)
print(counter_c)
print(1331643)
print("Ready.")

534403
287402
509838
1331643
Ready.


# Cross-validation, machine learning model and training with different hyperparameters

Parameters to try with GridSearchCV:

* hidden_layer_sizes -> ith number = number of neurons in ith layer
* activation
* solver
* learning_rate
* learning_rate_init

In [16]:
print(datetime.datetime.now())

cross_valid = StratifiedKFold()

neural_network = MLPClassifier(activation='relu', solver='sgd', learning_rate = 'adaptive', max_iter=250, early_stopping=False)


params = {'hidden_layer_sizes': [(75,), (100,), (75, 100,), (100, 100,)],
          'solver': ['sgd', 'adam'],
          'learning_rate': ['constant', 'adaptive']}

grid = GridSearchCV(estimator=neural_network, cv=cross_valid, param_grid=params, return_train_score=True)
grid.fit(input_vectors_training, target_training)

# balanced data
# grid.fit(input_training_balanced, target_training_balanced)

print("Ready.")
print(datetime.datetime.now())

2021-10-19 13:47:23.016786


/home/annette/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/annette/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/annette/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/annette/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/annette/anaconda3/lib/python3.

Ready.
2021-10-21 01:16:47.365956


# Which one best?

In [17]:
score = grid.best_score_
best_params = grid.best_params_
results_grid = pnds.DataFrame(grid.cv_results_)
print(score)
print(best_params)
display(results_grid)

0.6594898281472524
{'hidden_layer_sizes': (75, 100), 'learning_rate': 'constant', 'solver': 'adam'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,param_solver,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,524.543223,59.988935,0.244642,0.001694,"(75,)",constant,sgd,"{'hidden_layer_sizes': (75,), 'learning_rate':...",0.633221,0.645848,...,0.646643,0.007355,15,0.652326,0.651381,0.649108,0.647021,0.650747,0.650117,0.001870
1,1243.578712,266.258944,1.122561,0.111490,"(75,)",constant,adam,"{'hidden_layer_sizes': (75,), 'learning_rate':...",0.640614,0.650004,...,0.652284,0.007099,12,0.662896,0.659492,0.656966,0.656003,0.655844,0.658240,0.002669
2,600.330851,40.879957,0.244578,0.001712,"(75,)",adaptive,sgd,"{'hidden_layer_sizes': (75,), 'learning_rate':...",0.633288,0.645728,...,0.646106,0.006931,16,0.651927,0.652787,0.651557,0.645229,0.648210,0.649942,0.002823
3,1061.302984,189.930799,1.075641,0.190022,"(75,)",adaptive,adam,"{'hidden_layer_sizes': (75,), 'learning_rate':...",0.641995,0.651375,...,0.652647,0.007244,11,0.662547,0.660232,0.660543,0.657249,0.651863,0.658487,0.003720
4,595.532107,68.510909,0.279106,0.002503,"(100,)",constant,sgd,"{'hidden_layer_sizes': (100,), 'learning_rate'...",0.634426,0.647012,...,0.648026,0.007594,14,0.652883,0.653054,0.652051,0.648906,0.650683,0.651515,0.001551
5,1734.607529,286.167326,1.713104,0.213115,"(100,)",constant,adam,"{'hidden_layer_sizes': (100,), 'learning_rate'...",0.642957,0.652558,...,0.656036,0.007775,5,0.664568,0.662781,0.663144,0.661083,0.661031,0.662522,0.001337
6,717.075464,3.788012,0.278666,0.001203,"(100,)",adaptive,sgd,"{'hidden_layer_sizes': (100,), 'learning_rate'...",0.635740,0.646621,...,0.648043,0.007152,13,0.656096,0.653914,0.653402,0.650619,0.651333,0.653073,0.001950
7,1460.068612,157.306175,1.709313,0.081332,"(100,)",adaptive,adam,"{'hidden_layer_sizes': (100,), 'learning_rate'...",0.641969,0.652114,...,0.655271,0.007969,6,0.665778,0.662371,0.661644,0.660197,0.660670,0.662132,0.001973
8,1051.300392,8.187935,0.376960,0.003411,"(75, 100)",constant,sgd,"{'hidden_layer_sizes': (75, 100), 'learning_ra...",0.642138,0.651964,...,0.653997,0.006851,9,0.665448,0.661349,0.660136,0.656874,0.659594,0.660680,0.002798
9,2789.512943,360.020488,1.865138,0.316569,"(75, 100)",constant,adam,"{'hidden_layer_sizes': (75, 100), 'learning_ra...",0.646017,0.657705,...,0.659490,0.007645,1,0.673442,0.675708,0.673398,0.669326,0.667717,0.671918,0.002941


# Saving the final model

In [19]:
# saving the model
dump(best_nn, '/home/annette/Data/Studium/Module/PBL/Neural_Networks/Best_ANN_0.659490.joblib')
print("Ready.")

Ready.
